# **Install LTN**

In [ ]:
!git clone https://github.com/logictensornetworks/logictensornetworks.git ltn

Cloning into 'ltn'...
remote: Enumerating objects: 339, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 339 (delta 47), reused 116 (delta 39), pack-reused 212
Receiving objects: 100% (339/339), 2.38 MiB | 9.49 MiB/s, done.
Resolving deltas: 100% (183/183), done.


In [ ]:
!pip3 install -e ltn/

Obtaining file:///content/ltn
  Found existing installation: logictensornetworks 0.1
    Can't uninstall 'logictensornetworks'. No files were found to uninstall.
  Running setup.py develop for logictensornetworks


# **Import libraries**

In [ ]:
import tensorflow as tf
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import logictensornetworks as ltn

ModuleNotFoundError: ignored

# **Load Data**

In [ ]:
# if running in google's colaboratory
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# path to trains-transformed.csv
path = '/content/drive/My Drive/IA/trains-transformed.csv'

In [ ]:
str_att = {
  'length': ['short', 'long'],
  'shape': ['closedrect', 'dblopnrect', 'ellipse', 'engine', 'hexagon',
          'jaggedtop', 'openrect', 'opentrap', 'slopetop', 'ushaped'],
  'load_shape': ['circlelod', 'hexagonlod', 'rectanglod', 'trianglod'],
  'Class_attribute': ['west','east']
}

def read_data(path=path):
  df = pd.read_csv(path, ',')

  for k in df:
    for att in str_att:
      if k.startswith(att):
        for i,val in enumerate(df[k]):
          if val in str_att[att]:
            df[k][i] = str_att[att].index(val)

  df.replace("\\0", 0, inplace=True)
  df.replace("None", -1, inplace=True)

  return df

df = read_data()
df = df.astype(float)
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Number_of_cars,Number_of_different_loads,num_wheels1,length1,shape1,num_loads1,load_shape1,num_wheels2,length2,shape2,num_loads2,load_shape2,num_wheels3,length3,shape3,num_loads3,load_shape3,num_wheels4,length4,shape4,num_loads4,load_shape4,Rectangle_next_to_rectangle,Rectangle_next_to_triangle,Rectangle_next_to_hexagon,Rectangle_next_to_circle,Triangle_next_to_triangle,Triangle_next_to_hexagon,Triangle_next_to_circle,Hexagon_next_to_hexagon,Hexagon_next_to_circle,Circle_next_to_circle,Class_attribute
0,5.0,4.0,2.0,1.0,6.0,3.0,2.0,2.0,0.0,8.0,1.0,3.0,3.0,1.0,6.0,1.0,1.0,2.0,0.0,6.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,4.0,3.0,2.0,0.0,9.0,1.0,3.0,2.0,0.0,7.0,1.0,2.0,2.0,0.0,0.0,2.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,4.0,2.0,2.0,0.0,6.0,1.0,0.0,2.0,0.0,4.0,1.0,3.0,3.0,1.0,0.0,1.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,5.0,2.0,2.0,0.0,7.0,1.0,3.0,2.0,0.0,1.0,1.0,3.0,2.0,0.0,2.0,1.0,2.0,2.0,0.0,6.0,1.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4.0,3.0,2.0,0.0,1.0,1.0,3.0,3.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,3.0,2.0,2.0,1.0,0.0,3.0,0.0,2.0,0.0,6.0,1.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,4.0,2.0,2.0,0.0,1.0,1.0,0.0,2.0,0.0,9.0,1.0,3.0,2.0,1.0,5.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,3.0,2.0,3.0,1.0,0.0,1.0,2.0,2.0,0.0,9.0,1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,5.0,2.0,2.0,0.0,7.0,1.0,0.0,2.0,1.0,5.0,1.0,2.0,2.0,0.0,6.0,1.0,2.0,2.0,0.0,7.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,3.0,1.0,2.0,0.0,9.0,1.0,2.0,2.0,1.0,6.0,2.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# **Preparação dos Dados**


In [ ]:
batch_size = 64
embedding_size = 5

In [ ]:
df = df.sample(frac=1) #shuffle
df_train = df[0:5]
df_test = df[5:]

,Number_of_cars,Number_of_different_loads,num_wheels1,length1,shape1,num_loads1,load_shape1,num_wheels2,length2,shape2,num_loads2,load_shape2,num_wheels3,length3,shape3,num_loads3,load_shape3,num_wheels4,length4,shape4,num_loads4,load_shape4,Rectangle_next_to_rectangle,Rectangle_next_to_triangle,Rectangle_next_to_hexagon,Rectangle_next_to_circle,Triangle_next_to_triangle,Triangle_next_to_hexagon,Triangle_next_to_circle,Hexagon_next_to_hexagon,Hexagon_next_to_circle,Circle_next_to_circle,Class_attribute
2,4.0,2.0,2.0,0.0,6.0,1.0,0.0,2.0,0.0,4.0,1.0,3.0,3.0,1.0,0.0,1.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
6,4.0,2.0,2.0,0.0,1.0,1.0,0.0,2.0,0.0,9.0,1.0,3.0,2.0,1.0,5.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,4.0,3.0,2.0,0.0,1.0,1.0,3.0,3.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
0,5.0,4.0,2.0,1.0,6.0,3.0,2.0,2.0,0.0,8.0,1.0,3.0,3.0,1.0,6.0,1.0,1.0,2.0,0.0,6.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
8,5.0,2.0,2.0,0.0,7.0,1.0,0.0,2.0,1.0,5.0,1.0,2.0,2.0,0.0,6.0,1.0,2.0,2.0,0.0,7.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
labels_train = df_train.pop("Class_attribute")
labels_test = df_test.pop("Class_attribute")

In [ ]:
ds_train = tf.data.Dataset.from_tensor_slices((df_train,labels_train)).batch(batch_size)
ds_test = tf.data.Dataset.from_tensor_slices((df_test,labels_test)).batch(batch_size)

# **Montagem do Modelo**

In [ ]:
CarCount = ltn.Predicate.MLP([embedding_size, embedding_size],hidden_layer_sizes=(8,8))
NoOfDiffLoads = ltn.Predicate.MLP([embedding_size, embedding_size],hidden_layer_sizes=(8,8))
NoWheels = [ltn.Predicate.MLP([embedding_size, embedding_size],hidden_layer_sizes=(8,8)) for i in range(4)]
Length = [ltn.Predicate.MLP([embedding_size, embedding_size],hidden_layer_sizes=(8,8)) for i in range(4)]
Shape = [ltn.Predicate.MLP([embedding_size, embedding_size],hidden_layer_sizes=(8,8)) for i in range(4)]
NoLoads = [ltn.Predicate.MLP([embedding_size, embedding_size],hidden_layer_sizes=(8,8)) for i in range(4)]
LoadShape = [ltn.Predicate.MLP([embedding_size, embedding_size],hidden_layer_sizes=(8,8)) for i in range(4)]

NextTo = [[ltn.Predicate.MLP([embedding_size],hidden_layer_sizes=(8,8)) for j in range(4)] for i in range(4)]

In [ ]:
nCarros = [[train for index, train in df_train.iterrows() if train["Number_of_cars"] == i] for i in range(6)]

loadCount = [[train for index, train in df_train.iterrows() if train["Number_of_different_loads"] == i] for i in range(5)]

wheelCount = [[[train for index, train in df_train.iterrows() if train["num_wheels"+j] == i] for i in range(5)] for j in range(5)]
length = [[[train for index, train in df_train.iterrows() if train["length"+j] == i] for i in range(2)] for j in range(5)]
shape = [[[train for index, train in df_train.iterrows() if train["shape"+j] == i] for i in range(8)] for j in range(5)]
loadInCar = [[[train for index, train in df_train.iterrows() if train["num_loads"+j] == i] for i in range(3)] for j in range(5)]
loadShape = [[[train for index, train in df_train.iterrows() if train["load_shape"+j] == i] for i in range(7)] for j in range(5)]

In [ ]:
formula_aggregator = ltn.fuzzy_ops.Aggreg_pMeanError(p=2)

@tf.function
def axioms(features, labels, training=False):

    for i in range(6):
      axioms.append(formula_aggregator(tf.stack(
          [CarCount(t, i) for i in nCarros[i]])))
      
    for i in range(5):
      axioms.append(formula_aggregator(tf.stack(
          [NoOfDiffLoads(t, i) for i in loadCount[i]])))
    
    for i in range(5):
      for j in range(5):
        axioms.append(formula_aggregator(tf.stack(
            [NoWheels(t, j) for i in wheelCount[i][j]])))
        
    for i in range(5):
      for j in range(2):
        axioms.append(formula_aggregator(tf.stack(
            [Length(t, j) for i in length[i][j]])))
        
    for i in range(5):
      for j in range(8):
        axioms.append(formula_aggregator(tf.stack(
            [Shape(t, j) for i in shape[i][j]])))
        
    for i in range(5):
      for j in range(3):
        axioms.append(formula_aggregator(tf.stack(
            [NoLoads(t, j) for i in loadInCar[i][j]])))
        
    for i in range(5):
      for j in range(7):
        axioms.append(formula_aggregator(tf.stack(
            [LoadShape(t, j) for i in loadShape[i][j]])))

    axioms = tf.stack(axioms)
    sat_level = formula_aggregator(axioms)
    return sat_level, axioms

In [ ]:
for features, labels in ds_test:
    print("Initial sat level %.5f"%axioms(features,labels)[0])
    break

# **Treinamento**

In [ ]:
trainable_variables = \
        CarCount.trainable_variables \
        + NoOfDiffLoads.trainable_variables \
        + NoWheels.trainable_variables \
        + Length.trainable_variables \
        + Shape.trainable_variables \
        + NoLoads.trainable_variables \
        + LoadShape.trainable_variables \
        + NextTo.trainable_variables

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

for epoch in range(2000):
    if 0 <= epoch < 400:
        p_exists = tf.constant(1.)
    else:
        p_exists = tf.constant(6.)
    with tf.GradientTape() as tape:
        loss_value = 1. - axioms(p_exists=p_exists)[0]
    grads = tape.gradient(loss_value, trainable_variables)
    optimizer.apply_gradients(zip(grads, trainable_variables))
    if epoch%200 == 0:
        print("Epoch %d: Sat Level %.3f"%(epoch, axioms(p_exists=p_exists)[0]))
print("Training finished at Epoch %d with Sat Level %.3f"%(epoch, axioms(p_exists=p_exists)[0]))